## Dữ Liệu Chạy Chương Trình

In [ ]:
import pandas as pd
import numpy as np
import os
import re
data_base = r"D:\18A1_Nhom_6_Case_11"
data_raw = os.path.join(data_base,"data_raw")
data_clean = os.path.join(data_base,"data_clean")

os.makedirs(data_clean, exist_ok=True)

df_customer = pd.read_csv(os.path.join(data_raw,'customer_profile.csv'))
df_consumption = pd.read_csv(os.path.join(data_raw,'consumption_log.csv'))
df_tariff = pd.read_csv(os.path.join(data_raw,'price_tariff.csv'))

df_customer_clean = pd.read_csv(os.path.join(data_clean,'customer_profile_clean.csv'))
df_consumption_clean = pd.read_csv(os.path.join(data_clean,'consumption_log_clean.csv'))
df_tariff_clean = pd.read_csv(os.path.join(data_clean,'price_tariff_clean.csv'))

Đã tải thành công 3 bảng dữ liệu: Khách hàng (40 dòng), Tiêu thụ (150 dòng), Bảng giá (5 dòng).

[customer_profile - 5 dòng đã làm sạch]:
  customer_id house_type    region
0      KH100    Chung cu    ha noi
1      KH101   Biệt thự     ha noi
2       KH102  chung cư     Hà Nội
3      KH103     Tập thể    TP.HCM
4       KH104    Tập thể  Bac Ninh

[consumption_log - 5 dòng đã làm sạch]:
  customer_id meter_id reading_date  consumption_kwh
0       kh136   ct016    27/01/2024             50.0
1       kh119   ct010    2024-03-25             75.0
2       KH126    CT999   06/03/2024            150.0
3       kh135    CT999   2024-03-31             75.0
4       kh136    CT999   2024-04-15            300.0

[price_tariff - Bảng giá đã làm sạch]:
  tariff_level  kwh_min kwh_max price_per_kwh
0           B1        0      50          1800
1           B2       51     100         2000 
2           B3      101  200kwh          2500
3           B4      201     300         3000 
4           B5      301

## NHIỆM VỤ 3

## Tính lượng điện tiêu thụ trung bình theo house_type × region

In [11]:
# 2. Hợp nhất Dữ liệu và Tính Chi phí 
df_complete = pd.merge(df_consumption, df_customer, on='customer_id', how='left')
df_complete['estimated_cost'] = 0


#Tính lượng điện tiêu thụ trung bình theo house_type × region
print("\n--- 3.1. TB Điện tiêu thụ (kWh) theo Loại nhà và Khu vực ---")
avg_consumption_by_type_region = df_complete.groupby(['house_type', 'region'])['consumption_kwh'].mean().round(2).reset_index()
avg_consumption_by_type_region.columns = ['Loại Nhà', 'Khu Vực', 'TB Tiêu Thụ (kWh)']
print(avg_consumption_by_type_region.sort_values(by='TB Tiêu Thụ (kWh)', ascending=False))



--- 3.1. TB Điện tiêu thụ (kWh) theo Loại nhà và Khu vực ---
     Loại Nhà    Khu Vực  TB Tiêu Thụ (kWh)
1   Biệt thự   bac  ninh             300.00
20   biet thu    Đà Nẵng             300.00
27   tap the     Đà Nẵng             210.00
22  chung cư      Hà Nội             179.00
19   biet thu     tp hcm             178.00
10    Tập thể   Bac Ninh             175.00
9    Nhà phố      TP.HCM             175.00
13    Tập thể     TP.HCM             169.00
7     NHA pho     Ha Noi             167.50
6    Chung cu    Đà Nẵng             156.67
21  chung cư    Bac Ninh             150.00
2   Biệt thự      ha noi             142.14
0   Biệt thự     Da Nang             141.67
8    Nhà phố     Da Nang             135.00
14    Tập thể  bac  ninh             135.00
23   tap the    Bac Ninh             122.00
17   biet thu     Hà Nội             118.12
5    Chung cu     Ha Noi             116.88
12    Tập thể     Hà Nội             108.33
25   tap the      ha noi              97.50
26   tap the  

In [12]:
print("\n--- 3.2. Tổng Điện năng và Tổng Tiền điện theo Khách hàng (Top 5 ) ---")
# Tính tổng điện năng và tổng tiền điện theo từng khách hàng
customer_agg = df_complete.groupby('customer_id').agg(
    Tổng_Điện_Năng=('consumption_kwh', 'sum'),
    Tổng_Tiền_Điện=('estimated_cost', 'sum'),
    Số_Kỳ_Ghi_Số=('reading_date', 'count')
).reset_index()
customer_agg['Tổng_Tiền_Điện'] = customer_agg['Tổng_Tiền_Điện'].round(0).astype(int)
print(customer_agg.sort_values(by='Tổng_Tiền_Điện', ascending=False).head(5))


--- 3.2. Tổng Điện năng và Tổng Tiền điện theo Khách hàng (Top 5 ) ---
  customer_id  Tổng_Điện_Năng  Tổng_Tiền_Điện  Số_Kỳ_Ghi_Số
0       KH102           450.0               0             2
1       KH104           350.0               0             2
2       KH105           300.0               0             1
3       KH108           570.0               0             6
4       KH112            50.0               0             2


## Xác định nhóm house_type có mức tiêu thụ cao nhất.

In [13]:
#Xác định nhóm house_type có mức tiêu thụ cao nhất
print("\n--- 3.3. Xếp hạng Loại nhà theo Tổng Điện năng tiêu thụ ---")
house_type_rank = df_complete.groupby('house_type')['consumption_kwh'].sum().sort_values(ascending=False).to_frame(name='Tổng Điện Năng Tiêu Thụ (kWh)')
print(house_type_rank)
highest_type = house_type_rank.index[0]
print(f"\n=> **Kết luận:** Nhóm **{highest_type}** là nhóm tiêu thụ điện năng lớn nhất về tổng thể.")


--- 3.3. Xếp hạng Loại nhà theo Tổng Điện năng tiêu thụ ---
            Tổng Điện Năng Tiêu Thụ (kWh)
house_type                               
tap the                            3890.0
Tập thể                            2815.0
biet thu                           2610.0
Biệt thự                           2370.0
Chung cu                           2345.0
chung cư                           2090.0
Nhà phố                            1375.0
NHA pho                             670.0

=> **Kết luận:** Nhóm **tap the ** là nhóm tiêu thụ điện năng lớn nhất về tổng thể.


## Phân tích chênh lệch tiêu thụ giữa các region.

In [14]:
#Phân tích chênh lệch tiêu thụ giữa các region
print("\n--- 3.4. Phân tích Chênh lệch và Độ phân tán Tiêu thụ theo Khu vực ---")
region_stats = df_complete.groupby('region')['consumption_kwh'].agg(
    TB_Tiêu_Thụ='mean',
    Độ_Lệch_Chuẩn='std'
).round(2).reset_index()
region_stats.columns = ['Khu Vực', 'TB Tiêu Thụ (kWh)', 'Độ Lệch Chuẩn (kWh)']
region_stats = region_stats.sort_values(by='TB Tiêu Thụ (kWh)', ascending=False)
print(region_stats)
highest_region_avg = region_stats.iloc[0]['TB Tiêu Thụ (kWh)']
lowest_region_avg = region_stats.iloc[-1]['TB Tiêu Thụ (kWh)']
diff = highest_region_avg - lowest_region_avg
print(f"\n=> **Phân tích:** Khu vực **{region_stats.iloc[0]['Khu Vực']}** có mức tiêu thụ trung bình cao nhất.")
print(f"=> Sự chênh lệch giữa khu vực cao nhất và thấp nhất là: **{diff:.2f} kWh/kỳ**.")



--- 3.4. Phân tích Chênh lệch và Độ phân tán Tiêu thụ theo Khu vực ---
     Khu Vực  TB Tiêu Thụ (kWh)  Độ Lệch Chuẩn (kWh)
4     TP.HCM             171.67               101.43
5  bac  ninh             162.50               111.52
8    Đà Nẵng             145.91                99.96
3     Hà Nội             145.71                93.49
7     tp hcm             132.27                89.87
0   Bac Ninh             122.92                76.80
2     Ha Noi             122.19                77.18
1    Da Nang             116.07                82.40
6     ha noi             115.83                66.77

=> **Phân tích:** Khu vực **TP.HCM** có mức tiêu thụ trung bình cao nhất.
=> Sự chênh lệch giữa khu vực cao nhất và thấp nhất là: **55.84 kWh/kỳ**.
